In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

from torch.utils.data import DataLoader


from pitch_tracker.utils import dataset
from pitch_tracker.utils.constants import (F_MIN, HOP_LENGTH, N_FFT, N_MELS,
                                           PICKING_FRAME_SIZE,
                                           PICKING_FRAME_STEP,
                                           PICKING_FRAME_TIME, SAMPLE_RATE,
                                           STEP_FRAME, STEP_TIME, WIN_LENGTH,
                                           N_CLASS, )
from pitch_tracker.utils import files

/Users/tien.d/workspace/GITHUB/mono_pitch_tracker/medleydb/medleydb/__init__.py:69: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  INST_TAXONOMY = yaml.load(fhandle)
/Users/tien.d/workspace/GITHUB/mono_pitch_tracker/medleydb/medleydb/__init__.py:77: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  MIXING_COEFFICIENTS = yaml.load(fhandle)


In [4]:
import os
import pandas as pd

from pitch_tracker.utils import files, dataset
from sklearn.model_selection import train_test_split


DATASET_DIR = '../content/pickled_database/'
SPLIT_BY_DIR = True


if SPLIT_BY_DIR:
    dataset_paths = list(files.list_folder_paths_in_dir(DATASET_DIR))
    train_set, validation_set = train_test_split(dataset_paths, test_size=0.40, random_state=1, shuffle=True)
    validation_set, test_set = train_test_split(validation_set, test_size=0.50, random_state=1, shuffle=True)
    print(f'train_song_set: {len(train_set)}')
    print(f'validation_song_set: {len(validation_set)}')
    print(f'test_song_set: {len(test_set)}')
else:
    dataset_paths = files.list_all_file_paths_in_dir(DATASET_DIR)
    train_set, validation_set = train_test_split(dataset_paths, test_size=0.40, random_state=1, shuffle=True)
    validation_set, test_set = train_test_split(validation_set, test_size=0.50, random_state=1, shuffle=True)
    print(f'train_set: {len(train_set)}')
    print(f'validation_set: {len(validation_set)}')
    print(f'test_set: {len(test_set)}')

train_song_set: 64
validation_song_set: 21
test_song_set: 22


In [5]:
train_dataset = dataset.AudioDataset(train_set)
validation_set = dataset.AudioDataset(validation_set)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
validation_dataloader = DataLoader(validation_set, batch_size=8, shuffle=True)

In [7]:
feature, label = next(iter(train_dataloader))

In [11]:
print(feature.shape)

torch.Size([8, 1, 1050, 88])


/Users/tien.d/opt/anaconda3/envs/thesis/lib/python3.9/site-packages/torch/_tensor_str.py:115: UserWarning: MPS: nonzero op is supported natively starting from macOS 13.0. Falling back on CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/Indexing.mm:217.)
  nonzero_finite_vals = torch.masked_select(


tensor([[[[-0.5613, -0.5219, -0.5741,  ...,  0.2112,  0.3167,  0.2146],
          [-0.5423, -0.4031, -0.2304,  ...,  0.3055,  0.4130,  0.2017],
          [-0.7108, -0.4375, -0.2274,  ...,  0.3723,  0.7768,  0.3443],
          ...,
          [-0.1147, -0.7266, -0.7043,  ...,  0.7677,  0.6574,  0.6462],
          [-0.3576, -0.4508, -0.2811,  ...,  0.7485,  0.6462,  0.6462],
          [-0.1582,  0.0437,  0.1221,  ...,  0.7809,  0.6462,  0.6462]],

         [[ 0.2057,  0.2612,  0.4637,  ...,  0.2484,  0.1754,  0.4030],
          [ 0.4885,  0.6602,  0.7403,  ...,  0.2776,  0.1582,  0.1450],
          [ 0.1313,  0.4378,  0.7379,  ...,  0.2917,  0.3853, -0.0293],
          ...,
          [-0.4067, -0.2190, -0.1114,  ..., -0.1980, -0.2134, -0.2093],
          [ 0.2246,  0.3363,  0.3845,  ..., -0.1608, -0.2093, -0.2093],
          [-0.0144, -0.0396, -0.1506,  ..., -0.1827, -0.2093, -0.2093]],

         [[-0.4276, -0.3287, -0.2649,  ..., -0.1737, -0.1909, -0.1294],
          [-0.2544,  0.0752,  